In [4]:
import json
import pandas as pd
from datasets import Dataset

def load_data(file_path):
    """
    JSON 데이터를 불러와 Pandas DataFrame으로 변환
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
        
    data = []
    for item in json_data['data']:
        if 'text' in item and 'keyword' in item:
            data.append({
                'text': item['text'],
                'keyword': item['keyword']
            })
    
    df = pd.DataFrame(data)
    print(df.info())
    # None 값을 빈 문자열로 대체
    df = df.fillna('')
    return df


# JSON 파일 경로
file_path_location = "/home/yjtech/Desktop/LLM/Data/smell_keyword/location_data.json"
location_df = load_data(file_path_location)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     3000 non-null   object
 1   keyword  3000 non-null   object
dtypes: object(2)
memory usage: 47.0+ KB
None


In [7]:
from sklearn.model_selection import train_test_split

# 데이터를 70:30 비율로 나누기
def split_data(df, train_ratio = 0.8):
    """
    DataFrame을 train과 val로 나눔
    """
    train_df, val_df = train_test_split(df, train_size = train_ratio, random_state = 42, shuffle = True)
    return train_df, val_df

# train, val로 나누기
train_location_df, val_location_df = split_data(location_df)

train_location_df.to_csv('/home/yjtech/Desktop/LLM/Data/smell_keyword/train_location_df.csv', index = False)
val_location_df.to_csv('/home/yjtech/Desktop/LLM/Data/smell_keyword/val_location_df.csv', index = False)

# 나눈 데이터 확인
print("Location")
print("Train Data:")
print(train_location_df.head())
print("\nValidation Data:")
print(val_location_df.head())

# 데이터 크기 확인
print(f"Train size: {len(train_location_df)}, Validation size: {len(val_location_df)}")

Location
Train Data:
                                                   text     keyword
642   저희 포항 공원에서 바람에 실린 꽃 향기 덥게 문제로 불편을 겪고 있습니다. 9월 ...       포항 공원
700   호미곶등대에서 7월 25일 오전 8시부터 9시에 발생한 얼음이 녹은 냄새 짧게 강풍...       호미곶등대
226   최근 2월 17일 저녁에 불법 폐기물 처리장에서 지속적으로 나는 냄새 병적으로 문제...  불법 폐기물 처리장
1697  농촌 테마파크에서 가축의 배설물 처리 냄새 자극이 지속적으로 반복되는 작물 오염가 ...     농촌 테마파크
1010  고무 공장에서 8개월 전 오후 1시부터 2시에 발생한 화학적 물질 증발 냄새 뚜렷하...       고무 공장

Validation Data:
                                                   text      keyword
1801  안녕하세요. 시큼한 냄새 동물의 체취처럼 문제로 염전 인근 목장에서 12월 26일 ...     염전 인근 목장
1190  가정용 전자기기 공장에서 5월 9일 오후 4시부터 5시에 발생한 포장지 처리 냄새 ...  가정용 전자기기 공장
1817  종축 시험장에서 저장된 배설물 냄새 고약하게 퍼지는 악취와 건강 문제가 발생하여 불...       종축 시험장
251   안녕하세요. 찌든 냄새 강제적으로 압도하는 문제로 포항 근교 농촌에서 10개월 전 ...     포항 근교 농촌
2505  지하철 공사 현장에서 28일 전 오후 4시부터 5시에 발생한 석탄 가루 냄새이 잦은...    지하철 공사 현장
Train size: 2400, Validation size: 600


In [8]:
import json
import pandas as pd
from datasets import Dataset

train_location_df = pd.read_csv('/home/yjtech/Desktop/LLM/Data/smell_keyword/train_location_df.csv')
val_location_df = pd.read_csv('/home/yjtech/Desktop/LLM/Data/smell_keyword/val_location_df.csv')



train_location_data_dict = train_location_df.to_dict(orient='list')
location_train_dataset = Dataset.from_dict(train_location_data_dict)
location_train_dataset
print(len(location_train_dataset))

val_location_data_dict = val_location_df.to_dict(orient='list')
location_val_dataset = Dataset.from_dict(val_location_data_dict)
print(len(location_val_dataset))

2400
600


In [ ]:
import os
import torch
from tqdm import tqdm
from typing import Dict
import time
from datetime import datetime
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AdamW,
    DataCollatorForSeq2Seq,
)

from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import GradScaler, autocast
from konlpy.tag import Mecab
import re

class CustomKeyBERTTrainer:
    def __init__(self, model_name: str, **kwargs):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {self.device}")
        if self.device == "cuda":
            print(f"GPU Model: {torch.cuda.get_device_name(0)}")
            print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(self.device)
        self.optimizer = AdamW(self.model.parameters(), lr=kwargs.get("learning_rate", 1e-4))
        self.max_length = kwargs.get("max_length", 128)
        self.training_args = kwargs
        self.save_dir = kwargs.get("save_dir", "./location_models")
        self.mecab = Mecab()  # MeCab 초기화
        self.best_model_path = os.path.join(self.save_dir, "pytorch_model.bin")
        self.tokenizer_path = self.save_dir
        
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'epoch_times': [],
            'best_epoch': 0
        }

        os.makedirs(self.save_dir, exist_ok=True)
        
    def _mecab_tokenize(self, text: str) -> str:
        """
        MeCab을 사용하여 입력 텍스트에서 명사, 형용사, 부사를 추출
        """
        pos_tags = self.mecab.pos(text)  # 품사 태그 추출
        # 원하는 품사 필터링: 명사(NNG, NNP), 형용사(VA), 부사(MAG, MAJ)
        keyword = [
            word for word, pos in pos_tags if pos in ("NNG", "NNP", "VA", "MAG", "MAJ")
        ]
        return " ".join(keyword)  # 추출한 단어를 공백으로 연결하여 반환
    
    def _extract_context(self, text: str) -> str:
        """
        문맥 정보를 추출하여 키워드와 함께 사용
        """
        # 문장의 앞뒤 문맥 문장 추출
        sentences = re.split(r'[.!?]+', text)
        context = " ".join(sentences[-2:] + sentences[:2])
        
        # MeCab을 사용하여 문맥 내 명사, 형용사, 부사 추출
        context_keywords = self._mecab_tokenize(context)
        
        return context_keywords
    
    def preprocess_data(self, examples: Dict) -> Dict:
        # MeCab 형태소 분석과 품사 필터링 적용
        examples["text"] = [self._mecab_tokenize(text) for text in examples["text"]]
        
        # 문맥 정보 추출
        examples["context"] = [self._mecab_tokenize(self._extract_context(text)) for text in examples["text"]]
        
        # 입력과 레이블 구성
        inputs = [f"키워드 추출: {text} {context}" for text, context in zip(examples["text"], examples["context"])]
        model_inputs = self.tokenizer(
            inputs,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors='pt'  # 텐서 변환을 DataCollator에 맡김
        )

        # labels 처리
        labels = []
        for keyword in examples["keyword"]:
            if isinstance(keyword, list):  # 키워드가 리스트인 경우
                labels.append(", ".join(keyword) if keyword else "")
            else:  # 단일 키워드 문자열인 경우
                labels.append(keyword if keyword else "")

        with self.tokenizer.as_target_tokenizer():
            tokenized_labels = self.tokenizer(
                labels,
                max_length=self.max_length,
                padding="max_length",
                truncation=True,
                return_tensors="pt"  # 텐서 변환
            )

        # -100으로 패딩 토큰을 마스킹
        labels = tokenized_labels["input_ids"]
        for i in range(len(labels)):
            for j in range(len(labels[i])):
                if labels[i][j] == self.tokenizer.pad_token_id:
                    labels[i][j] = -100

        model_inputs["labels"] = labels

        return model_inputs
        
    def save_model_and_tokenizer(self, epoch=None, is_best=False):
        """
        최고 성능 모델만 저장하고 이전 모델을 삭제
        """
        if is_best:
            # 이전 최고 모델 디렉토리 삭제
            if os.path.exists(self.best_model_path):
                print(f"Deleting previous best model at {self.best_model_path}")
                os.system(f"rm -rf {self.best_model_path}")
            
            # 새로운 최고 모델 저장
            save_path = os.path.join(self.save_dir, f"best_model_epoch_{epoch}")
            os.makedirs(save_path, exist_ok=True)
            self.model.save_pretrained(save_path)
            self.tokenizer.save_pretrained(save_path)
            torch.save(self.history, os.path.join(save_path, 'training_history.pt'))
            print(f"New best model saved at {save_path}")

            # 최고 모델 경로 업데이트
            self.best_model_path = save_path

    def calculate_metrics(self, predictions, labels):
        predictions = torch.argmax(predictions, dim=-1)
        correct = (predictions == labels).masked_fill(labels == -100, 0)
        accuracy = correct.sum().item() / (labels != -100).sum().item()
        return accuracy



    def train(self, train_dataset, valid_dataset=None):
        
        start_time = time.time()
        print(f"\nStarting training at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Training parameters:")
        print(f"- Batch size: {self.training_args['batch_size']}")
        print(f"- Learning rate: {self.training_args.get('learning_rate', '2e-5')}")
        print(f"- Max length: {self.max_length}")
        print(f"- Number of epochs: {self.training_args['num_epochs']}")
        print(f"- Training samples: {len(train_dataset)}")
        if valid_dataset:
            print(f"- Validation samples: {len(valid_dataset)}")
        print("\n" + "="*50 + "\n")
        
        # 데이터셋 전처리
        print("Preprocessing training data...")
        train_dataset = train_dataset.map(
            self.preprocess_data,
            batched=True,
            remove_columns=train_dataset.column_names,
            desc="Training data processing"
        )

        if valid_dataset is not None:
            valid_dataset = valid_dataset.map(
                self.preprocess_data,
                batched=True,
                remove_columns=valid_dataset.column_names,
                desc="Validation data processing"
            )

        # DataCollator 설정
        data_collator = DataCollatorForSeq2Seq(
            tokenizer=self.tokenizer,
            model=self.model,
            padding=True,
            return_tensors="pt"
        )

        # DataLoader 설정 (num_workers=0으로 변경하여 멀티프로세싱 관련 오류 방지)
        train_dataloader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=self.training_args["batch_size"],
            shuffle=True,
            collate_fn=data_collator,
            num_workers=0,
            pin_memory=True
        )

        if valid_dataset is not None:
            valid_dataloader = torch.utils.data.DataLoader(
                valid_dataset,
                batch_size=self.training_args["batch_size"],
                shuffle=False,
                collate_fn=data_collator,
                num_workers=0,
                pin_memory=True
            )
        best_val_loss = float('inf')
        early_stopping_counter = 0
        early_stopping_patience = self.training_args.get('patience', 3)

        for epoch in range(self.training_args["num_epochs"]):
            epoch_start_time = time.time()
            
            self.model.train()
            epoch_loss = 0
            epoch_accuracy = 0
            train_steps = 0
            
            progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}")
            batch_losses = []
            batch_accuracies = []
            
            for batch_idx, batch in enumerate(progress_bar):
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels,
                )
                
                loss = outputs.loss # 손실 계산
                accuracy = self.calculate_metrics(outputs.logits, labels)
                
                loss.backward()# 손실 역전파 
                self.optimizer.step() # 가중치 업데이트
                self.optimizer.zero_grad() # 그래디언트 초기화

                batch_losses.append(loss.item())
                batch_accuracies.append(accuracy)
                
                current_loss = np.mean(batch_losses[-100:])
                current_accuracy = np.mean(batch_accuracies[-100:])
                progress_bar.set_postfix({
                    'loss': f'{current_loss:.4f}',
                    'accuracy': f'{current_accuracy:.4f}',
                    'batch': f'{batch_idx + 1}/{len(train_dataloader)}'
                })

            avg_train_loss = np.mean(batch_losses)
            avg_train_accuracy = np.mean(batch_accuracies)

            if valid_dataset is not None:
                self.model.eval()
                val_losses = []
                val_accuracies = []

                print("\nRunning validation...")
                with torch.no_grad():
                    for batch in tqdm(valid_dataloader, desc="Validating"):
                        input_ids = batch["input_ids"].to(self.device)
                        attention_mask = batch["attention_mask"].to(self.device)
                        labels = batch["labels"].to(self.device)

                        outputs = self.model(
                            input_ids=input_ids,
                            attention_mask=attention_mask,
                            labels=labels,
                        )
                        
                        loss = outputs.loss
                        accuracy = self.calculate_metrics(outputs.logits, labels)
                        
                        val_losses.append(loss.item())
                        val_accuracies.append(accuracy)

                avg_val_loss = np.mean(val_losses)
                avg_val_accuracy = np.mean(val_accuracies)

                if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    early_stopping_counter = 0
                    self.history['best_epoch'] = epoch + 1
                    print(f"\nNew best validation loss: {best_val_loss:.4f}")
                    self.save_model_and_tokenizer(epoch + 1, is_best=True)  # 최고 모델만 저장
                else:
                    early_stopping_counter += 1


            epoch_time = time.time() - epoch_start_time
            self.history['epoch_times'].append(epoch_time)
            self.history['train_loss'].append(avg_train_loss)
            if valid_dataset is not None:
                self.history['val_loss'].append(avg_val_loss)

            # Print epoch summary
            print(f"\nEpoch {epoch + 1} Summary:")
            print(f"Time taken: {epoch_time:.2f} seconds")
            print(f"Average training loss: {avg_train_loss:.4f}")
            print(f"Training accuracy: {avg_train_accuracy:.4f}")
            if valid_dataset is not None:
                print(f"Validation loss: {avg_val_loss:.4f}")
                print(f"Validation accuracy: {avg_val_accuracy:.4f}")
                print(f"Best validation loss so far: {best_val_loss:.4f}")
                print(f"Early stopping counter: {early_stopping_counter}/{early_stopping_patience}")

            if early_stopping_counter >= early_stopping_patience:
                print("\nEarly stopping triggered.")
                break

    def predict(self, text: str) -> str:
        """모델 추론"""
        context_keywords = self._extract_context(text)
        inputs = self.tokenizer(
            f"키워드 추출: {self._mecab_tokenize(text)} {context_keywords}",
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
        ).to(self.device)

        outputs = self.model.generate(
            inputs["input_ids"], max_length=self.max_length, num_beams=5
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def _normalize_text(self, text: str) -> str:
        return text.strip()

In [20]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 멀티프로세싱 경고 방지

trainer = CustomKeyBERTTrainer(
    model_name="facebook/bart-base", # t5-base     skt/kobart-base-v2
    max_length=128,
    learning_rate=1e-4,
    batch_size=8,
    num_epochs=10,
    gradient_accumulation_steps=8,
    patience=3 # 몇 에폭마다 체크포인트 저장할지
)

if __name__ == "__main__":
    torch.cuda.empty_cache()  # GPU 메모리 초기화
    trainer.train(location_train_dataset, location_val_dataset)

Using device: cpu

Starting training at: 2024-12-06 04:58:16
Training parameters:
- Batch size: 8
- Learning rate: 0.0001
- Max length: 128
- Number of epochs: 10
- Training samples: 2400
- Validation samples: 600


Preprocessing training data...


Training Epoch 1:   0%|          | 0/300 [00:00<?, ?it/s]


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`context` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def test_model(text, saved_model_path, type):

    # 모델과 토크나이저 불러오기
    tokenizer = AutoTokenizer.from_pretrained(saved_model_path, local_files_only=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(saved_model_path, local_files_only=True).to('cpu')
    
    inputs = tokenizer(
        f"키워드 추출: {text}",
        return_tensors="pt",
        max_length=128,
        truncation=True
    ).to("cpu")

    outputs = model.generate(
        inputs["input_ids"],
        max_length=128,
        num_beams=5,
        length_penalty=0.7,
        repetition_penalty=1.2,
        early_stopping=True
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("원래 문장: ", text)
    print(f"{type}키워드 추출: {result}")
    

text = '저희 포스코에서 아침에 쓰레기장 인근 불쾌한 냄새로 인해 불편을 겪고 있어 연락드립니다. 감사합니다. 도시 환경 관리팀입니다. 포스코에서 발생한 산성 냄새 문제와 관련해 상담 드리겠습니다. 최근 포스코에서 1시 산성 냄새이 축축하게 나고 있어 주변 환경 개선 필요로 큰 불편을 겪고 있습니다.\
    현재 포스코에서 발생한 쓰레기장 인근 불쾌한 냄새 문제에 대해 조치 중입니다. 1시에 완료될 예정입니다. 네, 처리가 되도록 부탁드립니다. 네, 알겠습니다. 잘 부탁드립니다. 감사합니다. 빠르게 처리하겠습니다.'
        
test_model(text, '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/smell_keyword/location_models/best_model_epoch_3', 'location')


원래 문장:  저희 포스코에서 아침에 쓰레기장 인근 불쾌한 냄새로 인해 불편을 겪고 있어 연락드립니다. 감사합니다. 도시 환경 관리팀입니다. 포스코에서 발생한 산성 냄새 문제와 관련해 상담 드리겠습니다. 최근 포스코에서 1시 산성 냄새이 축축하게 나고 있어 주변 환경 개선 필요로 큰 불편을 겪고 있습니다.    현재 포스코에서 발생한 쓰레기장 인근 불쾌한 냄새 문제에 대해 조치 중입니다. 1시에 완료될 예정입니다. 네, 처리가 되도록 부탁드립니다. 네, 알겠습니다. 잘 부탁드립니다. 감사합니다. 빠르게 처리하겠습니다.
location키워드 추출: 저, 희, 스, 코, 에, 서


: 